# Full order model  for importance sampling

## This script does the following 

* Models the linear-linear coupled problem with varying gaussian source term
* Solves the full-order model using domain decomposition

In [ ]:
# print("Generating snapshots")

class Leftup(SubDomain):
    def inside(self, x, on_boundary):
        return (between(x[1], (0.5,1.0)) and near(x[0],0.0))

class Rightup(SubDomain):
    def inside(self, x, on_boundary):
        return (between(x[1], (0.5,1.0)) and near(x[0],1.0))

class Leftdown(SubDomain):
    def inside(self, x, on_boundary):
        return (between(x[1], (0.0,0.5)) and near(x[0],0.0))
    
class Rightdown(SubDomain):
    def inside(self, x, on_boundary):
        return (between(x[1], (0.0,0.5)) and near(x[0],1.0))
    
class Bottom(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], 0.0)

class Top(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], 1.0)

class Obstacle(SubDomain):
    def inside(self, x, on_boundary):
        return (between(x[1], (0.0, 0.5)) and between(x[0], (0.0, 1.0)))
    
start = time.time()
    
# Initialize sub-domain instances
leftup = Leftup()
leftdown = Leftdown()
rightup = Rightup()
rightdown = Rightdown()
bottom = Bottom()
top = Top()
obstacle = Obstacle()

# Define mesh
gridsize = 100
mesh = UnitSquareMesh(gridsize, gridsize)

# Initialize mesh function for interior domains
domains = MeshFunction("size_t", mesh,2)
domains.set_all(0)
obstacle.mark(domains, 1)

# Initialize mesh function for boundary domains
boundaries = MeshFunction("size_t", mesh,1)
boundaries.set_all(0)

leftup.mark(boundaries, 1)
leftdown.mark(boundaries, 2)

rightup.mark(boundaries,3)
rightdown.mark(boundaries, 4)

top.mark(boundaries, 5)
bottom.mark(boundaries, 6)

# Define input data
g_T = Expression("1",degree=1)
g_B = Expression("-1",degree=1)
f1 = Constant(0)

# Define function space and basis functions
V = FunctionSpace(mesh, "Lagrange", 1)
u = TrialFunction(V)
v = TestFunction(V)

# Define Dirichlet boundary conditions at left and right boundaries
u0 = Expression("1", degree=2)

# Define new measures associated with the interior domains and
# exterior boundaries
dx = Measure("dx")(subdomain_data = domains)
ds = Measure("ds")(subdomain_data = boundaries)

dofs_x = V.tabulate_dof_coordinates().reshape((-1, 2))

# Set seed for reproducability
random.seed(11)

para1 = [0, 1]
para2 = [0, 0.5]
para3 = [0.001, 0.1]

params = 3 # number of parameters
# nsamples = 252 # number of samples including test samples
# ntest = 1 # number of test samples

nh = (gridsize+1)**2 # dimension of space

# Create arrays for vectors F
size = int(gridsize*gridsize/2+gridsize/2)

F_2I_array = np.zeros((nsamples,gridsize+1))
F_I_array = np.zeros((nsamples,gridsize+1))
F_2_array = np.zeros((nsamples,size))

# Initialize 
mu = np.zeros((nsamples,params))
S = np.zeros((nh,nsamples))

# Select indices that correspond to the dofs on domain 1, 2 and interface respectively
rows_domain1 = np.where(dofs_x[:,1]>0.5)[0]
rows_domain2 = np.where(dofs_x[:,1]<0.5)[0]
rows_interface = np.where(dofs_x[:,1]==0.5)[0]

ndofs_U1 = len(rows_domain1)
ndofs_UI = len(rows_interface)
ndofs_U2 = len(rows_domain2)

S1 = np.zeros((ndofs_U1,nsamples))
SI = np.zeros((ndofs_UI,nsamples))
S2 = np.zeros((ndofs_U2,nsamples))

# Initialize matrices to rearrange matrix A
E_1 = sps.csc_matrix((np.ones(len(rows_domain1)),(rows_domain1,np.arange(len(rows_domain1)))), shape=(V.dim(),len(rows_domain1)))
E_2 = sps.csc_matrix((np.ones(len(rows_domain2)),(rows_domain2,np.arange(len(rows_domain2)))), shape=(V.dim(),len(rows_domain2)))
E_interface = sps.csc_matrix((np.ones(len(rows_interface)),(rows_interface,np.arange(len(rows_interface)))), shape=(V.dim(),len(rows_interface)))

# Define dirichlet boundary conditions
bcs1 = [DirichletBC(V, u0, boundaries, 1),
DirichletBC(V, u0, boundaries, 3)]

bcs2 = [DirichletBC(V, u0, boundaries, 2),
DirichletBC(V, u0, boundaries, 4)]


FEMstart = time.time()


for i in range(nsamples):
    
    print(i+1, "snapshots computed,",nsamples-i-1,"snapshots to go...", end="\r")

    # Define randomness in parameters
    
    par1 = np.random.normal(mu_star[0], 0.5)
    par2 = np.random.normal(mu_star[1], 0.5)
    par3 = np.random.normal(mu_star[2], 0.1)
    
    # Store the parameters
    mu[i] = [par1, par2, par3]
    
#     print(mu[i])

    # Define source term on domain 2
    f2 = Expression("100*exp(-0.5*pow((pow(x[0]-par1,2)+pow(x[1]-par2,2))/(par3),2))",degree=2,par1=par1, par2=par2, par3=par3)

    # Define variational form
    if i == 0:
        F1 = (inner(grad(u), grad(v))*dx(0) - f1*v*dx(0) - g_T*v*ds(5))
        a_1, L1 = lhs(F1), rhs(F1)
        
    F2 = (inner(grad(u), grad(v))*dx(1) - f2*v*dx(1) - g_B*v*ds(6))
    a_2, L2 = lhs(F2), rhs(F2)
    
    # Transform UFL object to a sparse csc
    a1, f_1 = UFL_to_csc(a_1,L1,bcs1)
    a2, f_2 = UFL_to_csc(a_2,L2,bcs2)
    
    if i == 0:
        
        # Create block matrices
        A_11 = E_1.T*a1*E_1
        A_22 = E_2.T*a2*E_2

        A_II1 = E_interface.T*a1*E_interface
        A_II2 = E_interface.T*a2*E_interface
        A_II = A_II1+A_II2

        A_1I = E_1.T*a1*E_interface
        A_2I = E_2.T*a2*E_interface
        A_I1 = E_interface.T*a1*E_1
        A_I2 = E_interface.T*a2*E_2
        F_1 = f_1[rows_domain1]
        F_1I = f_1[rows_interface]


    # do vector F in similar way
    
    F_2 = f_2[rows_domain2]
    F_2I = f_2[rows_interface]
    F_I = F_1I+F_2I

    # Put A and F back together
    A_final = sps.bmat([[A_11, None, A_1I],[None, A_22,A_2I],[A_I1,A_I2,A_II]]).tocsc()
    F_final = sps.hstack([F_1,F_2,F_I]).tocsc()
    
    # Store F
    F_2I_array[i] = F_2I

    F_I_array[i] = F_I
    F_2_array[i] = F_2
    
    # Solve
    U = sps.linalg.spsolve(A_final,F_final.T)
    
    if i == 0:
        FEMend = time.time()
        FEMtime = FEMend - FEMstart     
    
    # Store
    S[:,i] = U
    
    S1[:,i] = U[0:ndofs_U1]
    SI[:,i] = U[-ndofs_UI:]
    S2[:,i] = U[ndofs_U1:ndofs_U1+ndofs_U2]
    
#     print(np.mean(S2[:,i]))
 
# print("\n Finished.")
